###### Data notebook overview

In this notebook I will import data, save data to csv files for use elsewhere, and verify that there is sufficient data for prediction by Elo, Glicko-2, and rating points.

###### Attributions

[data source](<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" href="http://purl.org/dc/dcmitype/Dataset" property="dct:title" rel="dct:type">Tennis databases, files, and algorithms</span> by <a xmlns:cc="http://creativecommons.org/ns#" href="http://www.tennisabstract.com/" property="cc:attributionName" rel="cc:attributionURL">Jeff Sackmann / Tennis Abstract</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.<br />Based on a work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/JeffSackmann" rel="dct:source">https://github.com/JeffSackmann</a>)

An efficient solution to creating one df from multiple files is from [Sid](https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe) 

###### Imports

In [1]:
import time
from datetime import datetime, timedelta
import pandas as pd
import glob
import sys 
import numpy as np
sys.path.append('..')

Since the data license is non-commercial, this is a proof-of-concept project and cannot be immediately applied to commercial use.

###### Read in data

In [2]:
# read in data
atp_matches = glob.glob('../tennis_atp/atp_matches*.csv')

# separate out by match category
matches_tour = [file for file in atp_matches if 
                       len(file) == len('../tennis_atp/atp_matches_2019.csv')]
matches_qual_chall = glob.glob('../tennis_atp/atp_matches_qual_chall_*.csv')
matches_futures = glob.glob('../tennis_atp/atp_matches_futures_*.csv')
rankings = glob.glob('../tennis_atp/atp_rankings_*.csv')
players = glob.glob('../tennis_atp/atp_players.csv')
matches_amateur = glob.glob('../tennis_atp/atp_matches_amateur.csv')

# singles matches file list
singles_matches = matches_qual_chall + matches_futures + matches_tour + \
matches_amateur
# Combined DataFrames
singles_matches_df = pd.concat((pd.read_csv(f, low_memory=False,parse_dates = 
                                            ['tourney_date'], 
                                            infer_datetime_format=True) 
                                for f in singles_matches), 
                               ignore_index=True)
# select columns to be used in determining glicko2 ratings
matches_glicko2 = singles_matches_df[['tourney_date','winner_id','loser_id',
                                      'tourney_id']]
# Separate Dataframes
players_df = pd.read_csv(players[0],low_memory=False) 
rankings_df = pd.concat((pd.read_csv(ranks) for ranks in rankings))
matches_qual_chall_df = pd.concat((pd.read_csv(matches,low_memory=False) for matches in 
                                   matches_qual_chall))
matches_futures_df = pd.concat((pd.read_csv(matches,low_memory=False) for matches in 
                                matches_futures))
matches_tour_df = pd.concat((pd.read_csv(matches,low_memory=False) for matches in matches_tour))
matches_amateur_df = pd.read_csv(matches_amateur[0],low_memory=False)

In [3]:
singles_matches_df.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,1999-460,Heilbronn CH,Carpet,32,C,1999-01-25,1,103082,NaN,Q,...,NaN,NaN,NaN,NaN,NaN,NaN,199.0,228.0,52.0,818.0
1,1999-460,Heilbronn CH,Carpet,32,C,1999-01-25,2,102615,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,121.0,394.0,120.0,401.0
2,1999-460,Heilbronn CH,Carpet,32,C,1999-01-25,3,103451,NaN,WC,...,NaN,NaN,NaN,NaN,NaN,NaN,440.0,64.0,119.0,403.0
3,1999-460,Heilbronn CH,Carpet,32,C,1999-01-25,4,102107,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,146.0,321.0,108.0,447.0
4,1999-460,Heilbronn CH,Carpet,32,C,1999-01-25,5,102652,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,142.0,324.0,86.0,584.0


In [4]:
# sort the rows by tourney_date
matches_glicko2 = matches_glicko2.sort_values(by = 'tourney_date')

# adjust the index
matches_glicko2.index = [x for x in range(matches_glicko2.shape[0])]

# save to csv for import by other notebooks
matches_glicko2.to_csv('../Data/matches_glicko2.csv',index=False)

In [5]:
# sort the rows by tourney_date
singles_matches_df.sort_values(by = 'tourney_date', inplace = True) 

# adjust the index
singles_matches_df.index = [x for x in range(singles_matches_df.shape[0])]

In [13]:
# save to csv for import by other notebooks
n = singles_matches_df.shape[0]//3
singles_matches_df[0:n].to_csv('../Data/singles_matches_df_0.csv',index=False)
singles_matches_df[n:n*2].to_csv('../Data/singles_matches_df_1.csv',index=False)
singles_matches_df[n*2:].to_csv('../Data/singles_matches_df_2.csv',index=False)


In [7]:
singles_matches_df[['winner_id','loser_id']]

,winner_id,loser_id
0,113987,114149
1,113987,113999
2,113987,113963
3,113963,114158
4,113999,113953
...,...,...
876973,207987,208519
876974,144932,209977
876975,209955,209942
876976,209079,210696


All matches have a winner and loser id, which indicates that all the datapoints can be used for determining the elo ranking.  There is a good amount of standing rank data, suggesting that this data can be used for testing the relative strength of standard ranking vs elo rating in predicting outcomes.

In [8]:
# NaN analysis adapted from code by James Larkin
(1-singles_matches_df.isnull().mean()).mul(100).sort_values(ascending=False)

tourney_id            100.000000
winner_id             100.000000
round                 100.000000
tourney_name          100.000000
loser_name            100.000000
loser_id              100.000000
winner_name           100.000000
best_of               100.000000
match_num             100.000000
draw_size             100.000000
tourney_date          100.000000
tourney_level         100.000000
winner_ioc             99.998860
loser_ioc              99.992474
score                  99.983466
surface                99.548905
winner_age             98.426300
winner_hand            97.757754
loser_hand             97.575424
loser_age              96.703680
winner_rank            87.879057
loser_rank             80.534859
winner_rank_points     80.096308
loser_rank_points      73.733891
winner_ht              61.427767
loser_ht               49.171815
winner_seed            42.112915
loser_entry            28.936644
loser_seed             21.519240
w_SvGms                21.290272
l_SvGms   

In [9]:
singles_matches_df[singles_matches_df['tourney_date']>
                   '1999-01-25']['loser_rank_points'].notnull().mean(),\
singles_matches_df[singles_matches_df['tourney_date']>
                   '1999-01-25']['winner_rank_points'].notnull().mean()

(0.8770289179104478, 0.9525636327292111)

Tourney Date, winner id and loser id have no missing values.  Thus there is no filtering by row needed for rating system analysis. winner_rank_points data is available for 80% of the observations.  loser_rank_points is available for 73% of the observations.  loser_rank_points is available for 87% of the matches after 1999-01-25
and winner_rank_points is available for 95% of the matches after 1999-01-25.  This suggests there is ample ranking points data available in the modern era with which to investigate predictive value of ranking points.